## Introduction

This kernel shows how to use NBSVM (Naive Bayes - Support Vector Machine) to create a strong baseline for the [Toxic Comment Classification Challenge](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge) competition. NBSVM was introduced by Sida Wang and Chris Manning in the paper [Baselines and Bigrams: Simple, Good Sentiment and Topic Classiﬁcation](https://nlp.stanford.edu/pubs/sidaw12_simple_sentiment.pdf). In this kernel, we use sklearn's logistic regression, rather than SVM, although in practice the two are nearly identical (sklearn uses the liblinear library behind the scenes).

If you're not familiar with naive bayes and bag of words matrices, I've made a preview available of one of fast.ai's upcoming *Practical Machine Learning* course videos, which introduces this topic. Here is a link to the section of the video which discusses this: [Naive Bayes video](https://youtu.be/37sFIak42Sc?t=3745).

In [1]:
import pandas as pd, numpy as np
import re
import string
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

In [2]:
X_train = pd.read_csv('data/toxic_train.tsv', sep='\t')
X_test = pd.read_csv('data/toxic_test.tsv', sep='\t')

In [3]:
X_train.shape

(143717, 4)

In [4]:
X_test.shape

(15969, 4)

In [5]:
y_train = X_train.toxic
y_test = X_test.toxic

The length of the comments varies a lot.

In [6]:
lens = X_train.comment.str.len()
lens.mean(), lens.std(), lens.max()

(373.85100874781654, 562.22421959175108, 5000.0)

We'll create a list of all the labels to predict, and we'll also create a 'none' label so we can see how many comments have no labels. We can then summarize the dataset.

In [9]:
COMMENT = 'comment'
X_train[COMMENT].fillna("unknown", inplace=True)
X_test[COMMENT].fillna("unknown", inplace=True)

## Building the model

We'll start by creating a *bag of words* representation, as a *term document matrix*. We'll use ngrams, as suggested in the NBSVM paper.

In [10]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

It turns out that using TF-IDF gives even better priors than the binarized features used in the paper. I don't think this has been mentioned in any paper before, but it improves leaderboard score from 0.59 to 0.55.

In [11]:
n = X_train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(X_train[COMMENT])
test_term_doc = vec.transform(X_test[COMMENT])

This creates a *sparse matrix* with only a small number of non-zero elements (*stored elements* in the representation  below).

In [12]:
trn_term_doc, test_term_doc

(<143717x354556 sparse matrix of type '<class 'numpy.float64'>'
 	with 12639180 stored elements in Compressed Sparse Row format>,
 <15969x354556 sparse matrix of type '<class 'numpy.float64'>'
 	with 1386292 stored elements in Compressed Sparse Row format>)

Here's the basic naive bayes feature equation:

In [13]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [14]:
x = trn_term_doc
test_x = test_term_doc

Fit a model for one dependent at a time:

In [15]:
def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=4, dual=True)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [16]:
label_cols = ['toxic']
preds = np.zeros((len(X_test), len(label_cols)))

for i, j in enumerate(label_cols):
    print('fit', j)
    m,r = get_mdl(X_train[j])
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

fit toxic


# Evaluation

In [19]:
y_true = X_test.as_matrix(columns=['toxic'])
log_loss(y_true, preds[:,0]) 

0.28937769880165842

In [20]:
accuracy_score(y_true, preds[:, 0].round())

0.8924165570793412